<a href="https://colab.research.google.com/github/lmedinelli/UAIAI/blob/master/mod2_heuristics/Tarea2_Constrain_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Universidad Adolfo Ibañez](https://www.uai.cl/assets/themes/uai-central/img/logo.svg)

---

# UAI : Magister / Diploma AI 2020

---

## Mini-proyecto 2 - Programación Basada en Restricciones

### Descripción Problema a resolver

La evaluación consiste en realizar un modelo
que resuelva el problema de asignar trabajadores
a vuelos en una compañía aérea. Para lo que tenemos los siguientes datos:

1. Hay 20 trabajadores
+ Azafatos: Tom, David, Jeremy, Ron, Joe, Bill, Fred, Bob,
Mario, Ed
+ Azafatas: Carol, Janet, Tracy, Marilyn, Carolyn, Cathy,
Inez, Jean, Heather, Juliet
2. Hay 10 vuelos (la numeración es importante)
3. Cada vuelo necesita el siguiente número de
trabajadores, azafatos y azafatas.

![](https://drive.google.com/uc?export=view&id=1cHJBX0X42Auo0EwdRfY_xUByM_V71m9K)

4. Además, cada vuelo necesita al menos una persona
que sepa francés, una persona que sepa español y
una persona que sepa alemán.
* Saben francés: Inez, Bill, Jean, Juliet
* Saben español: Tom, Jeremy, Mario, Cathy, Juliet
* Saben alemán: Bill, Fred, Joe, Mario, Marilyn, Inez, Heather

5. Si un trabajador trabaja un vuelo, no puede
trabajar ninguno de los siguientes dos vuelos
(ejemplo, si Mario trabaja en el vuelo 1, no
puede trabajar en el vuelo 2 ni en el vuelo 3).

---

### Descripción **Objetivo**

* El objetivo es encontrar una asignación de
trabajadores a vuelos.

* Pregunta extra: ¿Se podría hacer con menos
trabajadores?

# Solución

XXXXXX Descripción de como se piensa resolver el problema XXXXXxxxx

## Importación de paquetes y creación de variables

In [1]:
pip install ortools

     |████████████████████████████████| 29.5MB 135kB/s 
     |████████████████████████████████| 1.3MB 46.1MB/s 
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


In [1]:
from ortools.constraint_solver import pywrapcp

In [ ]:
#help(pywrapcp)

In [2]:
solver = pywrapcp.Solver("Tripulacion")

In [4]:
 nombres = [
      "Tom", "David", "Jeremy", "Ron", "Joe", "Bill", "Fred", "Bob", "Mario",
      "Ed", "Carol", "Janet", "Tracy", "Marilyn", "Carolyn", "Cathy", "Inez",
      "Jean", "Heather", "Juliet"
  ]

In [5]:
print(nombres)

['Tom', 'David', 'Jeremy', 'Ron', 'Joe', 'Bill', 'Fred', 'Bob', 'Mario', 'Ed', 'Carol', 'Janet', 'Tracy', 'Marilyn', 'Carolyn', 'Cathy', 'Inez', 'Jean', 'Heather', 'Juliet']


In [6]:
personas = len(nombres)
print(personas)

20


Se establece la matriz de atributos que corresponden a toda la tripulación, de modo de poder clasificar y clusterizar los ambitos que tenemos de trabajo de la tripulación.

In [7]:
atributos = [
      #  Azafatos, Azafatas, Frances, Español, Aleman
      [1, 0, 0, 1, 0],  # Tom     = 1
      [1, 0, 0, 0, 0],  # David   = 2
      [1, 0, 0, 1, 0],  # Jeremy  = 3
      [1, 0, 0, 0, 0],  # Ron     = 4
      [1, 0, 0, 0, 1],  # Joe     = 5
      [1, 0, 1, 0, 1],  # Bill    = 6
      [1, 0, 0, 0, 1],  # Fred    = 7
      [1, 0, 0, 0, 0],  # Bob     = 8
      [1, 0, 0, 1, 1],  # Mario   = 9
      [1, 0, 0, 0, 0],  # Ed      = 10
      [0, 1, 0, 0, 0],  # Carol   = 11
      [0, 1, 0, 0, 0],  # Janet   = 12
      [0, 1, 0, 0, 0],  # Tracy   = 13
      [0, 1, 0, 1, 1],  # Marilyn = 14
      [0, 1, 0, 0, 0],  # Carolyn = 15
      [0, 1, 0, 1, 0],  # Cathy   = 16
      [0, 1, 1, 0, 1],  # Inez    = 17
      [0, 1, 1, 0, 0],  # Jean    = 18
      [0, 1, 0, 0, 1],  # Heather = 19
      [0, 1, 1, 1, 0]  # Juliet  = 20
  ]

Se forma la siguiente tabla requerida por las condiciones del problema generado correspondiente a la tripulación requerida por fila (cada fila es un vuelo de la tabla del encabezado):

  + Numero Trabajadores     : Numero total de trabajadores requeridos en vuelo
  + Azafatos  : Azafatos requeridos
  + Azafatas : Azafatas requeridas
  + Frances    : Cuantos requeridos que sepan frances
  + Español   : Cuantos que sepan español son requeridos
  + Aleman    : Cuantos que sepan Aleman son requeridos

In [21]:

tripulacion_requerida = [
                         [4, 1, 1, 1, 1, 1],  # Vuelo 1
                         [5, 1, 1, 1, 1, 1],  # Vuelo 2
                         [5, 1, 1, 1, 1, 1],  # Vuelo 3
                         [6, 2, 2, 1, 1, 1],  # Vuelo 4
                         [7, 3, 3, 1, 1, 1],  # Vuelo 5
                         [4, 1, 1, 1, 1, 1],  # Vuelo 6
                         [5, 1, 1, 1, 1, 1],  # Vuelo 7
                         [6, 1, 1, 1, 1, 1],  # Vuelo 8
                         [6, 2, 2, 1, 1, 1],  # Vuelo 9
                         [7, 3, 3, 1, 1, 1]   # Vuelo 10
                         ]

numero_vuelos = len(tripulacion_requerida)  # numero de vuelos
tripulacion_requerida[9][0]

7

Se genera un diccionario de tripulación con llave una tupla que tiene el valor del vuelo en el primer elemento y de la cantidad de tripulacióón en el segundo y el elemnto valor corresponde a un solver que puede ser 0 o 1.

Tambien se crea una variable numero de trabajadores que contieen un solver que corresponde a una variable entera que puede ir desde 1 a numero de personas, llamada num_trab

In [9]:
tripulacion = {}
for i in range(mumero_vuelos):
  for j in range(personas):
    # tripilacion es un diccionario que las llaves son una tupla entre el numero de vuelos
    # y valor es un solver que tiene una varible entera que va de 0 a 1, por lo que indicara si esa persona 
    # de la tripulación pertenece al vuelo ej. si llave es (0,17) y el valor es 1, implica que en vuelo 1 va Jean
    tripulacion[(i, j)] = solver.IntVar(0, 1, "tripulacion[%i,%i]" % (i, j)) 

tripulacion_f = [ tripulacion[(i, j)] for i in range(mumero_vuelos) for j in range(personas)]

  # numero de personas trabajando
num_trab = solver.IntVar(1, personas, "num_trab")

In [10]:
type(tripulacion)

dict

In [13]:
#tripulacion

In [12]:
num_trab

num_trab(1..20)

## Creación de restricciones

In [18]:
# numero de personas trabajando usando linked lists
solver.Add(num_trab == solver.Sum([
      solver.IsGreaterOrEqualCstVar( solver.Sum([tripulacion[(f, p)] for f in range(numero_vuelos)]), 1) for p in range(personas)
  ]))


In [24]:
 for f in range(numero_vuelos):
    # tamaño de la tripulacion ej [4, 1, 1, 1, 1, 1],  # Vuelo 1, donde son 4
    tmp = [tripulacion[(f, i)] for i in range(personas)]
    #print(tmp)
    solver.Add(solver.Sum(tmp) == tripulacion_requerida[f][0])
    # atributos y requerimientos
    for j in range(5): # 4 columnas de atributos requeridos
      # el producto entre los atributos de una caracteristica especifica (ej frances) por la tripulacion 
      # de un vuelo f  
      tmp = [atributos[i][j] * tripulacion[(f, i)] for i in range(personas)]
      solver.Add(solver.Sum(tmp) >= tripulacion_requerida[f][j + 1])

In [26]:
 # restriccion de descanso en vuelos (no se puede trabajar en los dos vuelos siguientes)
 for f in range(numero_vuelos - 2):
   for i in range(personas):
     solver.Add(tripulacion[f, i] + tripulacion[f + 1, i] + tripulacion[f + 2, i] <= 1)

In [29]:
  # extra : todos deben trabajar en al menos dos de los vuelos
  # for i in range(personas):
  #     [solver.Add(solver.Sum([tripulacion[f,i] for f in range(num_vuelos)]) >= 2) ]

# Solución y Búsqueda

In [34]:
solucion = solver.Assignment()
solucion.Add(tripulacion_f)
solucion.Add(num_trab)

# Revisar parametros del solver de assignacióón para determinar reducción de personas
db = solver.Phase(tripulacion_f, solver.CHOOSE_MIN_SIZE,
                    solver.ASSIGN_MIN_VALUE)

# Resultados

In [35]:

sols = 5 # determine el numero de soluciones a deplegar
solver.NewSearch(db)
num_solutions = 0
while solver.NextSolution():
  num_solutions += 1
  print("Solucion #%i" % num_solutions)
  print("Número Trabajadores:", num_trab.Value())
  for i in range(numero_vuelos):
    for j in range(personas):
      print(tripulacion[i, j].Value(), end=" ")
    print()
  print()

  print("Vuelos:")
  for flight in range(numero_vuelos):
    print("Vuelo", flight, "personas:", end=" ")
    for person in range(personas):
      if tripulacion[flight, person].Value() == 1:
        print(nombres[person], end=" ")
    print()
  print()

  print("Tripulacion:")
  for person in range(personas):
    print("%-10s Vuelos" % nombres[person], end=" ")
    for flight in range(numero_vuelos):
      if tripulacion[flight, person].Value() == 1:
        print(flight, end=" ")
    print()
  print()

  if num_solutions >= sols:
    break
solver.EndSearch()

print()
print("Numero Soluciones Desplegadas:", num_solutions)
print("Fallos:", solver.Failures())
print("Ramas:", solver.Branches())
print("WallTime:", solver.WallTime())

Solucion #1
Número Trabajadores: 19
0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 
0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 0 
0 0 1 0 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 1 
0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0 0 
0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 
0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 1 1 0 0 0 
0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 
0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 

Vuelos:
Vuelo 0 personas: Ed Jean Heather Juliet 
Vuelo 1 personas: Mario Marilyn Carolyn Cathy Inez 
Vuelo 2 personas: Jeremy Bill Carol Janet Tracy 
Vuelo 3 personas: Fred Bob Ed Jean Heather Juliet 
Vuelo 4 personas: Ron Joe Mario Marilyn Carolyn Cathy Inez 
Vuelo 5 personas: Jeremy Bill Janet Tracy 
Vuelo 6 personas: Ed Carol Jean Heather Juliet 
Vuelo 7 personas: Bob Mario Marilyn Carolyn Cathy Inez 
Vuelo 8 personas: Jeremy Joe Bill Fred Janet Tracy 
Vuelo 9 personas: David Ron Ed Carol Jean Heather Juliet 

Tripulacion:
Tom        Vuelos 
D

# Conclusiones

# Integrantes